In [ ]:
#This is cleaned up copy of my LB score. 

#Thanks to following public Kernels:
#https://www.kaggle.com/adityaecdrid/pytorch-bert-end-to-end-with-cv
#https://www.kaggle.com/phoenix9032/pytorch-bert-plain
#https://www.kaggle.com/artgor/pytorch-approach . ##crawl USE 
#https://www.kaggle.com/manyregression/fastai-no-spacy-transformers-here  ##Post processing

#And this post
#https://www.kaggle.com/c/google-quest-challenge/discussion/123770

In [ ]:
import os
from os.path import join as path_join

data_dir = '../input/'

NETLOC_PATH = '../input/bert-use-netloc/unique_netloc.pickle'

USE_URL = '../input/universalsentenceencoder4/'
TRAIN_CSV_PATH = path_join(data_dir,'google-quest-challenge', 'train.csv')
TEST_CSV_PATH = path_join(data_dir, 'google-quest-challenge','test.csv')
POST_PROCESS = False

In [ ]:

import os
import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time

from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup, BertPreTrainedModel,
)
from os.path import join as path_join

from tqdm import tqdm
print(transformers.__version__)


#wiki/crawl
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle


def set_seeds(SEED=42):
  os.environ["PYTHONHASHSEED"] = str(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

SEED=2019
set_seeds(SEED)
HEAD_TAIL = True

## Get Crawl Embedding matrix

In [ ]:
# added preprocessing from https://www.kaggle.com/wowfattie/3rd-place/data

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
mispell_dict = {"aren't" : "are not", "aren  '  t" : "are not",
"can't" : "cannot", "can  '  t" : "cannot",
"couldn't" : "could not","couldn  '  t" : "could not",
"couldnt" : "could not",
"didn't" : "did not","didn  '  t" : "did not",
"doesn't" : "does not","doesn  '  t" : "does not",
"doesnt" : "does not", 
"don't" : "do not","don  '  t" : "do not",
"hadn't" : "had not","hadn  '  t" : "had not",
"hasn't" : "has not", "hasn  '  t" : "has not",
"haven't" : "have not", "haven  '  t" : "have not",
"havent" : "have not",  
"he'd" : "he would", "he  '  d" : "he would",
"he'll" : "he will","he  '  ll" : "he will",
"he's" : "he is","he  '  s" : "he is",
"i'd" : "I would","i  '  d" : "I would",
"i'd" : "I had","i  '  d" : "I had",
"i'll" : "I will",  "i  '  ll" : "I will",
"i'm" : "I am",  "i  '  m" : "I am",
"isn't" : "is not", "isn  '  t" : "is not",
"it's" : "it is",  "it  '  s" : "it is",
"it'll":"it will",   "it  '  ll":"it will",
"i've" : "I have",   "i  '  ve" : "I have",
"let's" : "let us",  "let  '  s" : "let us",
"mightn't" : "might not",  "mightn  '  t" : "might not",
"mustn't" : "must not","mustn  '  t" : "must not",
"shan't" : "shall not","shan  '  t" : "shall not",
"she'd" : "she would","she  '  d" : "she would",
"she'll" : "she will","she  '  ll" : "she will",
"she's" : "she is","she  '  s" : "she is",
"shouldn't" : "should not","shouldn  '  t" : "should not",
"shouldnt" : "should not",
"that's" : "that is","that  '  s" : "that is",
"thats" : "that is",
"there's" : "there is","there  '  s" : "there is",
"theres" : "there is",
"they'd" : "they would","they  '  d" : "they would",
"they'll" : "they will","they  '  ll" : "they will",
"they're" : "they are","they  '  re" : "they are",
"theyre":  "they are",
"they've" : "they have","they  '  ve" : "they have",
"we'd" : "we would","we  '  d" : "we would",
"we're" : "we are","we  '  re" : "we are",
"weren't" : "were not","weren  '  t" : "were not",
"we've" : "we have","we  '  ve" : "we have",
"what'll" : "what will","what  '  ll" : "what will",
"what're" : "what are","what  '  re" : "what are",
"what's" : "what is","what  '  s" : "what is",
"what've" : "what have","what  '  ve" : "what have",
"where's" : "where is","where  '  s" : "where is",
"who'd" : "who would","who  '  d" : "who would",
"who'll" : "who will","who  '  ll" : "who will",
"who're" : "who are","who  '  re" : "who are",
"who's" : "who is","who  '  s" : "who is",
"who've" : "who have","who  '  ve" : "who have",
"won't" : "will not","won  '  t" : "will not",
"wouldn't" : "would not","wouldn  '  t" : "would not",
"you'd" : "you would","you  '  d" : "you would",

"you'll" : "you will","you  '  ll" : "you will",
"you're" : "you are","you  '  re" : "you are",
"you've" : "you have","you  '  ve" : "you have",
"'re": " are",
"wasn't": "was not","wasn  '  t": "was not",
"we'll":" will","we  '  ll":" will",
"didn't": "did not","didn  '  t": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df



def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr



def build_matrix(embedding_path: str = '',
                 embedding_path_spellcheck: str = path_join(data_dir, 'wiki-news-300d-1M.vec'),
                 word_dict: dict = None, max_features: int = 100000,
                 embed_size: int= 300, ):

    # embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path, encoding='utf-8'))
    embedding_index = load_embeddings(embedding_path)

    nb_words = min(max_features, len(word_dict))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    unknown_words = []
    for word, i in word_dict.items():
        key = word
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.lower())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.upper())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.capitalize())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        unknown_words.append(key)

    print(f'{len(unknown_words) * 100 / len(word_dict):.4f}% words are not in embeddings')
    return embedding_matrix, nb_words, unknown_words

In [ ]:
test = pd.read_csv(TEST_CSV_PATH).fillna(' ')
train = pd.read_csv(TRAIN_CSV_PATH).fillna(' ')

In [ ]:
train = clean_data(train, ['answer', 'question_body', 'question_title'])
test = clean_data(test, ['answer', 'question_body', 'question_title'])

tokenizer = Tokenizer()
full_text = list(train['question_body']) + \
                       list(train['answer']) + \
                       list(train['question_title']) + \
                       list(test['question_body']) + \
                       list(test['answer']) + \
                       list(test['question_title'])
tokenizer.fit_on_texts(full_text)

embed_size=300
embedding_path = "/kaggle/input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl" #path_join(data_dir,'crawl-300d-2M.pkl')

embedding_matrix, nb_words, unknown_words = build_matrix(embedding_path,  path_join(data_dir,'wiki-news-300d-1M.vec') , tokenizer.word_index,
                                              100000, embed_size)

## USE Features

In [ ]:
import os
import re
import gc
import pickle  
import random
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from scipy.stats import spearmanr, rankdata
from os.path import join as path_join
from numpy.random import seed
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet
import re

seed(SEED)
random.seed(SEED)

In [ ]:
train = pd.read_csv(TRAIN_CSV_PATH)
test = pd.read_csv(TEST_CSV_PATH)

print(train.shape, test.shape)

In [ ]:
#If test has urls other than these 59 = we have a problem 

find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

with open(NETLOC_PATH, "rb") as input_file:
     unique_netloc = pickle.load(input_file)
        
diff1  = list(set(test.netloc.unique()).difference(set(unique_netloc))) #netlocs that are in test set but not in pickled netlocs.
#replace these with unkowns


unknownlist = ['unknown1','unknown2','unknown3', 'unknown4', 'unknown5', 'unknown6', 'unknown7', 'unknown8', 'unknown9',
              'unknown10', 'unknown11', 'unknown12', 'unknown13', 'unknown14', 'unknown15',  'unknown16', 'unknown17', 'unknown18', 
             'unknown19', 'unknown20', 'unknown21',  'unknown22', 'unknown23', 'unknown24',  'unknown25', 'unknown26', 'unknown27',
              'unknown28', 'unknown29', 'unknown30', 'unknown31', 'unknown32', 'unknown33',  'unknown34', 'unknown35', 'unknown36', 
              'unknown37', 'unknown38','unknown39',  'unknown40']

if len(diff1) > 50: 
    exit(1)

for i,k in enumerate(diff1): 
    idx = test[test.netloc==k].index
    test.iloc[idx,11] = unknownlist[i]
    
    
tmp = pd.DataFrame([['unknown1', 'LIFE_ARTS'],['unknown2', 'CULTURE'],['unknown3', 'SCIENCE'],
             ['unknown4', 'STACKOVERFLOW'],['unknown5', 'TECHNOLOGY'],['unknown6', 'LIFE_ARTS'],
              ['unknown7', 'CULTURE'],['unknown8', 'SCIENCE'],['unknown9', 'STACKOVERFLOW'],
              ['unknown10', 'TECHNOLOGY'],['unknown11', 'CULTURE'],['unknown12', 'SCIENCE'],
              ['unknown13', 'STACKOVERFLOW'],['unknown14', 'TECHNOLOGY'],['unknown15', 'LIFE_ARTS'],
              ['unknown16', 'LIFE_ARTS'],['unknown17', 'CULTURE'],['unknown18', 'SCIENCE'],
             ['unknown19', 'STACKOVERFLOW'],['unknown20', 'TECHNOLOGY'],['unknown21', 'LIFE_ARTS'],
              ['unknown22', 'CULTURE'],['unknown23', 'SCIENCE'],['unknown24', 'STACKOVERFLOW'],
              ['unknown25', 'TECHNOLOGY'],['unknown26', 'CULTURE'],['unknown27', 'SCIENCE'],
              ['unknown28', 'STACKOVERFLOW'],['unknown29', 'TECHNOLOGY'],['unknown30', 'LIFE_ARTS'] ,
                    ['unknown31', 'LIFE_ARTS'],['unknown32', 'CULTURE'],['unknown33', 'SCIENCE'],
             ['unknown34', 'STACKOVERFLOW'],['unknown35', 'TECHNOLOGY'],['unknown36', 'LIFE_ARTS'],
              ['unknown37', 'CULTURE'],['unknown38', 'SCIENCE'],['unknown39', 'STACKOVERFLOW'],
              ['unknown40', 'TECHNOLOGY']
             ],columns =  ['netloc', 'category'])

In [ ]:
features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])
merged = merged.append(tmp[-(40 - len(diff1)):])

ohe = OneHotEncoder()
ohe.fit(merged)

features_train = ohe.transform(train[features]).toarray()
features_test = ohe.transform(test[features]).toarray()
print(features_train.shape, features_test.shape)


In [ ]:
module_url = USE_URL #path_join(data_dir,'use/')
embed = hub.load(module_url)

In [ ]:

input_columns = ['question_title', 'question_body', 'answer']

embeddings_train = {}
embeddings_test = {}
for text in input_columns:
    print(text)
    #train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    #curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    #while ind*batch_size < len(train_text):
    #    curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
    #    ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    #embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
gc.collect()



In [ ]:
#512 columns for each row, each column i.e. Row 1 -> question_title -> 512 features
#len(embeddings_train),
len(embeddings_test), embeddings_test['question_title_embedding'].shape

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close() 

cuda.select_device(0)
cuda.close()
cuda.select_device(0)

## Distance features

In [ ]:
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)

# dist_features_train = np.array([
#     l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
#     l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
#     l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
#     cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
#     cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
#     cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding'])
# ]).T

dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T

#X_train = np.hstack([item for k, item in embeddings_train.items()] + [features_train, dist_features_train])
X_test = np.hstack([item for k, item in embeddings_test.items()] + [features_test, dist_features_test])
#y_train = train[targets].values

In [ ]:
del embeddings_test, features_test, dist_features_test
gc.collect()
torch.cuda.empty_cache()

#X_train.shape, 
X_test.shape

In [ ]:
max_len = 500
max_len_title = 30
train_question_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['question_body']), maxlen = max_len)
train_answer_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['answer']), maxlen = max_len)
train_title_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['question_title']), maxlen = max_len_title)
test_question_tokenized = pad_sequences(tokenizer.texts_to_sequences(test['question_body']), maxlen = max_len)
test_answer_tokenized = pad_sequences(tokenizer.texts_to_sequences(test['answer']), maxlen = max_len)
test_title_tokenized = pad_sequences(tokenizer.texts_to_sequences(test['question_title']), maxlen = max_len_title)

## Bert Model

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [ ]:
#Change the model to include only question+title embedding and answer+title embedding
LEN_USE_FEATURES = 1646 #1606 
class BertForSequenceClassification_v2Answer(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).

    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

    Examples::

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

    """

    def __init__(self, config,embedding_matrix=None):
        
        super(BertForSequenceClassification_v2Answer,self).__init__(config)
        self.num_labels = config.num_labels

        self.embedding = nn.Embedding(*embedding_matrix.shape) 
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)

        self.linear_q_add = nn.Linear(300, 128)
        self.linear_q_add1 = nn.Linear(128, self.config.num_labels)
       
        # MINE - works
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  ,config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels)
        self.classifier = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  , self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None, 
        question_data = None,
        answer_data = None,
        title_data = None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
 
        last_hidden_state = outputs[0][:,0,:].reshape((-1, 1, 768)) #CLS Token 
        pooled_output = outputs[1] 
        hidden_states = outputs[2]

        # MINE - works
        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))
        all_h = torch.cat([ h9, h10, h11, h12], 1)
        all_h_mean = torch.mean(all_h, 1).reshape((-1,1,768))
        mean_pool = torch.mean(torch.cat([last_hidden_state,all_h_mean],1),1)
        pooled_output = self.dropout(mean_pool)


        #Embedding 
        h_embedding_q = self.embedding(question_data)
        h_embedding_q = self.embedding_dropout(h_embedding_q)

        h_embedding_a = self.embedding(answer_data)
        h_embedding_a = self.embedding_dropout(h_embedding_a)

        h_embedding_t = self.embedding(title_data)
        h_embedding_t = self.embedding_dropout(h_embedding_t)

        add = torch.cat((h_embedding_t, h_embedding_q, h_embedding_a), 1) #h_embedding_a,
        add = self.linear_q_add(torch.mean(add, 1))
        add = self.linear_q_add1(add)

        x = torch.nn.ELU()(self.linear1(torch.cat([pooled_output,usefeatures,add],1)))
        logits = self.classifier(x)

        #logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
#Change the model to include only question+title embedding and answer+title embedding
LEN_USE_FEATURES = 1646 #1606 
class BertForSequenceClassification_v2Question(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).

    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

    Examples::

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

    """

    def __init__(self, config,embedding_matrix=None):
        
        #super(BertForSequenceClassification, self).__init__(config)
        super(BertForSequenceClassification_v2Question,self).__init__(config)
        self.num_labels = config.num_labels

        self.embedding = nn.Embedding(*embedding_matrix.shape) 
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)

        self.linear_q_add = nn.Linear(300, 128)
          
        self.linear_q_add1 = nn.Linear(128, self.config.num_labels)

        
        # MINE - works
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  ,config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels)
        self.classifier = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  , self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None, 
        question_data = None,
        answer_data = None,
        title_data = None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
     
        last_hidden_state = outputs[0][:,0,:].reshape((-1, 1, 768)) #CLS Token 
        pooled_output = outputs[1] 
        hidden_states = outputs[2]

        # MINE - works
        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))
        all_h = torch.cat([ h9, h10, h11, h12], 1)
        all_h_mean = torch.mean(all_h, 1).reshape((-1,1,768))
        mean_pool = torch.mean(torch.cat([last_hidden_state,all_h_mean],1),1)
        pooled_output = self.dropout(mean_pool)

        #Embedding 
        h_embedding_q = self.embedding(question_data)
        h_embedding_q = self.embedding_dropout(h_embedding_q)

        h_embedding_t = self.embedding(title_data)
        h_embedding_t = self.embedding_dropout(h_embedding_t)

        add = torch.cat((h_embedding_t,  h_embedding_q), 1) #h_embedding_a,
        add = self.linear_q_add(torch.mean(add,1))
        add = self.linear_q_add1(add)

        x = torch.nn.ELU()(self.linear1(torch.cat([pooled_output,usefeatures,add],1)))
        logits = self.classifier(x)
        
        #logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)
train = pd.read_csv(TRAIN_CSV_PATH) # path_join(data_dir,"train.csv"))
test = pd.read_csv(TEST_CSV_PATH) #path_join(data_dir, "test.csv"))

target_cols_questions = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written'] 

target_cols_answers =  ['answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [ ]:
# helpers
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

## Tokenize Answers

In [ ]:
def _trim_input_answers(title, question, answer, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            


        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        q_len_head = round(q_new_len/2)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/2)
        a_len_tail = -1* (a_new_len -a_len_head)        ## Head+Tail method .
        t = t[:t_new_len]
        if HEAD_TAIL :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            q = q[:q_new_len]
            a = a[:a_new_len] ## No Head+Tail ,usual processing
    
    return t, q, a



def _convert_to_bert_inputs_answers(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"] 

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]



def compute_input_arays_answers(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q, a  = instance.question_title,instance.question_body, instance.answer
        t, q, a  = _trim_input_answers(t, q, a, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs_answers(t,q, a,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

## Tokenize Questions

In [ ]:
def _trim_input(title, question, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=479):
    
    #3 + 479 + 30
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)
    
    if (t_len+q_len+3) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
            q_new_len = q_max_len
        else:
            t_new_len = t_max_len
            q_new_len = q_max_len
      
        if q_max_len > q_len:
            q_new_len = q_len
            
            
        if t_new_len+q_new_len+3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + q_new_len + 3)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        
    return t, q

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] 

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q  = instance.question_title, instance.question_body
        t, q  = _trim_input(t, q, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

In [ ]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, usefeatures,question_data, answer_data, title_data, lengths, labels = None):
        
        self.inputs = inputs
        self.usefeatures = usefeatures
        self.question_data = question_data
        self.answer_data = answer_data
        self.title_data = title_data
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths
       

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        usef            = self.usefeatures[idx,:]
        question_data   = self.question_data[idx,:]
        answer_data     = self.answer_data[idx,:]
        title_data      = self.title_data[idx,:]

        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments,  usef,  question_data, answer_data, title_data, labels, lengths
        return input_ids, input_masks, input_segments, usef,  question_data,answer_data, title_data,  lengths

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def predict_result_question(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), 21))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader),position=0, leave=True )
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                            usefeatures = x_batch[3].to(device),
                            question_data = x_batch[4].to(device),
                            answer_data = x_batch[5].to(device),
                            title_data = x_batch[6].to(device)
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output


def predict_result_answer(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), 9))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader),position=0, leave=True )
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                            usefeatures = x_batch[3].to(device),
                            question_data = x_batch[4].to(device),
                            answer_data = x_batch[5].to(device),
                            title_data = x_batch[6].to(device)
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output

In [ ]:
set_seeds(SEED)

question_tokenizer = BertTokenizer.from_pretrained("../input/bert-vocab1/vocab.txt")
input_categories_q = list(train.columns[[1,2]]); 

bert_model_config_question = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
bert_config_q = BertConfig.from_json_file(bert_model_config_question)
bert_config_q.output_hidden_states = True
bert_config_q.num_labels = 21

answer_tokenizer = BertTokenizer.from_pretrained("../input/bert-vocab1/vocab.txt")
input_categories_a = list(train.columns[[1,2,5]]); 

bert_model_config_answer = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
bert_config_a = BertConfig.from_json_file(bert_model_config_answer)
bert_config_a.output_hidden_states = True
bert_config_a.num_labels = 9

device = 'cuda' if torch.cuda.is_available() else 'cpu'

test_inputs_q = compute_input_arays(df =  test, columns = input_categories_q, tokenizer = question_tokenizer, max_sequence_length=512)
lengths_test_q = np.argmax(test_inputs_q[0] == 0, axis=1)
lengths_test_q[lengths_test_q == 0] = test_inputs_q[0].shape[1]


test_inputs_a = compute_input_arays_answers(df =  test, columns = input_categories_a, tokenizer = answer_tokenizer, max_sequence_length=512)
lengths_test_a = np.argmax(test_inputs_a[0] == 0, axis=1)
lengths_test_a[lengths_test_a == 0] = test_inputs_a[0].shape[1]

In [ ]:
NUM_FOLDS = 5   
BATCH_SIZE = 8
epochs = 4    
test_predictions_q = [] 
test_predictions_a = [] 
test_predictions = []

test_set_q = QuestDataset(inputs=test_inputs_q, 
                          usefeatures=torch.FloatTensor(X_test) , 
                          question_data =  torch.LongTensor(test_question_tokenized),
                          answer_data =  torch.LongTensor(test_answer_tokenized),
                          title_data =  torch.LongTensor(test_title_tokenized),
                          lengths=lengths_test_q, labels=None)
test_loader_q  = DataLoader(test_set_q, batch_size=32, shuffle=False)
result_q = np.zeros((len(test), 21))


test_set_a = QuestDataset(inputs=test_inputs_a, 
                          usefeatures=torch.FloatTensor(X_test) , 
                           question_data =  torch.LongTensor(test_question_tokenized),
                          answer_data =  torch.LongTensor(test_answer_tokenized),
                          title_data =  torch.LongTensor(test_title_tokenized),                          
                          lengths=lengths_test_a, labels=None)
test_loader_a  = DataLoader(test_set_a, batch_size=32, shuffle=False)
result_a = np.zeros((len(test), 9))


#y_train = train[target_cols].values # dummy
set_seeds(SEED)

In [ ]:
class_labels_dict = {'question_conversational': 2, 'question_expect_short_answer': 3, 'question_fact_seeking':4, 
 'question_has_commonly_accepted_answer':5,
 'question_multi_intent' : 8,
 'question_not_really_a_question':9,
 'question_opinion_seeking':10,
 'question_type_choice' :11, 
 'question_type_compare':12,
 'question_type_consequence': 13,
 'question_type_definition':14,
 'question_type_entity':15,
 'question_type_instructions':16,
 'question_type_procedure':17,
 'question_type_reason_explanation':18,
 'question_type_spelling':19,
 'answer_type_instructions':26,
 'answer_type_procedure':27,
 'answer_type_reason_explanation':28}

In [ ]:
BERT_SAVED_MODEL_ANSWERS = ['../input/bert-use-crawl2-dataset-answers/', 
                            '../input/bert-use-crawl-seed42-answers/',
                            '../input/bert-use-crawl-seed101-answers/',
                           '../input/bert-use-crawl2-dataset-seed1029-answers',
                           '../input/bert-use-crawl2-dataset-seed1977-answers']
                            
                            
BERT_SAVED_MODEL_QUESTIONS = ['../input/bert-use-crawl2-dataset-questions/'  ,
                              '../input/bert-use-crawl-seed42-questions/',
                            '../input/bert-use-crawl-seed101-questions/',
                             '../input/bert-use-crawl2-dataset-seed1029-questions',
                               '../input/bert-use-crawl2-dataset-seed1977-questions'
                             ]


for qa in range(len(BERT_SAVED_MODEL_ANSWERS)):    
    for f in range(NUM_FOLDS):

        model_q = BertForSequenceClassification_v2Question.from_pretrained('../input/pretrained-bert-models-for-pytorch/bert-base-uncased/', config=bert_config_q , embedding_matrix=embedding_matrix)
        model_q.zero_grad();
        model_q.to(device);
        torch.cuda.empty_cache()
        dict_load = torch.load(path_join(BERT_SAVED_MODEL_QUESTIONS[qa], 'Q_model_save_fold_{}.pt'.format(f)))
        dict_load['embedding.weight'] = torch.from_numpy(embedding_matrix)
        model_q.load_state_dict(dict_load)
        out_q = predict_result_question(model_q, test_loader_q)

        model_a = BertForSequenceClassification_v2Answer.from_pretrained('../input/pretrained-bert-models-for-pytorch/bert-base-uncased/',config=bert_config_a , embedding_matrix=embedding_matrix)
        model_a.zero_grad();
        model_a.to(device);
        torch.cuda.empty_cache()
        dict_load = torch.load(path_join(BERT_SAVED_MODEL_ANSWERS[qa], 'A_model_save_fold_{}.pt'.format(f)))
        dict_load['embedding.weight'] = torch.from_numpy(embedding_matrix)
        model_a.load_state_dict(dict_load)
        out_a = predict_result_answer(model_a, test_loader_a)
        preds = np.hstack((out_q,out_a))
        
        if POST_PROCESS:             
            # Round categories
            for l in class_labels_dict.keys():
                preds[:,class_labels_dict[l]] = pd.cut(preds[:,class_labels_dict[l]], train[l].nunique(), labels=train[l].value_counts().index.sort_values())


        test_predictions.append(preds)
    

In [ ]:
test_preds_bert = np.mean(test_predictions,axis=0)
idx = np.argwhere(test.category!='CULTURE').flatten()
arr = 0.0001 * np.clip(np.random.normal(size=len(idx)),0.1,0.99)
test_preds_bert[idx,19] = arr  

# class_labels = ['question_conversational',
#  'question_expect_short_answer',
#  'question_fact_seeking',
#  'question_has_commonly_accepted_answer',
#  'question_multi_intent',
#  'question_not_really_a_question',
#  'question_opinion_seeking',
#  'question_type_choice',
#  'question_type_compare',
#  'question_type_consequence',
#  'question_type_definition',
#  'question_type_entity',
#  'question_type_instructions',
#  'question_type_procedure',
#  'question_type_reason_explanation',
#  'question_type_spelling',
#  'answer_type_instructions',
#  'answer_type_procedure',
#  'answer_type_reason_explanation']


# submission = pd.read_csv('../input/google-quest-challenge/sample_submission.csv')
# submission.iloc[:,1:] = test_preds  

# # Round categories
# for l in class_labels:
#     submission[l] = pd.cut(submission[l], train[l].nunique(), labels=train[l].value_counts().index.sort_values())

# submission.to_csv('submission.csv', index=False)
# submission.head() 

# Roberta Inference

In [ ]:

ROBERTA_PRETRAINED = '../input/roberta-transformers-pytorch/roberta-base'


import os

import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time

#sys.path.insert(0, "gdrive/My Drive/kaggle/google-quest-challenge/")
sys.path.insert(0, data_dir)

#from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from transformers import (RobertaTokenizer, RobertaModel, BertPreTrainedModel, 
                          RobertaForSequenceClassification, RobertaConfig, AdamW, 
                          get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP )
#WEIGHTS_NAME, CONFIG_NAME, 

from os.path import join as path_join

from tqdm import tqdm
# import logging
# logging.basicConfig(filename=path_join('gdrive','My Drive','kaggle','submissions','roberta','robertamodel.log') ,level=logging.DEBUG)
    
print(transformers.__version__)

def set_seeds(SEED=42):
  os.environ["PYTHONHASHSEED"] = str(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

SEED=2019
set_seeds(SEED)
HEAD_TAIL = True

## Use the same USE + distance features created for Bert

## Model

In [ ]:
LEN_USE_FEATURES = 1646 #1606 


class RobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
      
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        
        return x

class RobertaForSequenceClassification_v2(BertPreTrainedModel):
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.RobertaConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import RobertaTokenizer, RobertaForSequenceClassification
        import torch

        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        """
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        sequence_output = outputs[0]

        pooled_output = outputs[1] 
        hidden_states = outputs[2]
        logits = self.classifier(sequence_output)   #I realize now that I am not using usefeatures here :( too late
        
        outputs = (logits,) + outputs[2:]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
train = pd.read_csv(TRAIN_CSV_PATH)
test = pd.read_csv( TEST_CSV_PATH)

target_cols_questions = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written'] 

target_cols_answers =  ['answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [ ]:
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "</s>":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

## Tokenize Answers

In [ ]:

def _trim_input_answers(title, question, answer, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        q_len_head = round(q_new_len/2)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/2)
        a_len_tail = -1* (a_new_len -a_len_head)        ## Head+Tail method .
        t = t[:t_new_len]
        if HEAD_TAIL :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            q = q[:q_new_len]
            a = a[:a_new_len] ## No Head+Tail ,usual processing
    
    return t, q, a



def _convert_to_bert_inputs_answers(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken =  ['<s> '] + title + ['</s> </s> '] + question + ['</s> </s> '] + answer + ['</s>'] 


    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]



def compute_input_arays_answers(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q, a  = instance.question_title,instance.question_body, instance.answer
        t, q, a  = _trim_input_answers(t, q, a, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs_answers(t,q, a,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

## Tokenize questions

In [ ]:
def _trim_input(title, question, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=479):
    
    #3 + 479 + 30
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)
    
    if (t_len+q_len+3) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
            q_new_len = q_max_len
        else:
            t_new_len = t_max_len
            q_new_len = q_max_len
      
        if q_max_len > q_len:
            q_new_len = q_len
            

        if t_new_len+q_new_len+3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + q_new_len + 3)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        
    return t, q

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ['<s>'] + title + ['</s> </s> '] + question + ['</s>'] 
    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q  = instance.question_title, instance.question_body
        t, q  = _trim_input(t, q, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

In [ ]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, usefeatures, lengths, labels = None):
        
        self.inputs = inputs
        self.usefeatures = usefeatures
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths
       

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        usef            = self.usefeatures[idx,:]
        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments,  usef, labels, lengths
        return input_ids, input_masks, input_segments, usef, lengths

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def predict_result_question(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), 21))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader),position=0, leave=True )
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                            usefeatures = x_batch[3].to(device)
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output


def predict_result_answer(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), 9))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader),position=0, leave=True )
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                            usefeatures = x_batch[3].to(device)                            
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output

In [ ]:
set_seeds(SEED)

question_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_PRETRAINED)
input_categories_q = list(train.columns[[1,2]]); 

robert_config_question =  RobertaConfig.from_pretrained(ROBERTA_PRETRAINED)
robert_config_question.num_labels = 21
robert_config_question.output_hidden_states = True 

answer_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_PRETRAINED)
input_categories_a = list(train.columns[[1,2,5]]); 

robert_config_answer =  RobertaConfig.from_pretrained(ROBERTA_PRETRAINED)
robert_config_answer.num_labels = 9
robert_config_answer.output_hidden_states = True 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
test_inputs_q = compute_input_arays(df =  test, columns = input_categories_q, tokenizer = question_tokenizer, max_sequence_length=512)
lengths_test_q = np.argmax(test_inputs_q[0] == 0, axis=1)
lengths_test_q[lengths_test_q == 0] = test_inputs_q[0].shape[1]


test_inputs_a = compute_input_arays_answers(df =  test, columns = input_categories_a, tokenizer = answer_tokenizer, max_sequence_length=512)
lengths_test_a = np.argmax(test_inputs_a[0] == 0, axis=1)
lengths_test_a[lengths_test_a == 0] = test_inputs_a[0].shape[1]

In [ ]:
NUM_FOLDS = 5   
BATCH_SIZE = 8
epochs = 5
test_predictions_q = [] 
test_predictions_a = [] 
test_predictions = []

test_set_q = QuestDataset(inputs=test_inputs_q, 
                          usefeatures=torch.FloatTensor(X_test), 
                          lengths=lengths_test_q, labels=None)
test_loader_q  = DataLoader(test_set_q, batch_size=32, shuffle=False)
result_q = np.zeros((len(test), 21))

test_set_a = QuestDataset(inputs=test_inputs_a, 
                          usefeatures=torch.FloatTensor(X_test),                  
                          lengths=lengths_test_a, labels=None)
test_loader_a  = DataLoader(test_set_a, batch_size=32, shuffle=False)
result_a = np.zeros((len(test), 9))

set_seeds(SEED)

In [ ]:
ROBERTA_SAVED_MODEL_ANSWERS = ['../input/roberta-dataset-answers/', 
                               '../input/roberta-seed-42-answers/' , 
                               '../input/roberta-dataset-seed101-answers/',
                              '../input/roberta-dataset-seed1029-answers'
                              ]

ROBERTA_SAVED_MODEL_QUESTIONS = ['../input/roberta-dataset-questions/',
                                '../input/roberta-seed-42-questions/',
                                 '../input/roberta-dataset-seed101-questions/',
                                '../input/roberta-dataset-seed1029-questions']  

    
for qa in range(len(ROBERTA_SAVED_MODEL_ANSWERS)):
    for f in range(NUM_FOLDS):

        model_q = RobertaForSequenceClassification_v2.from_pretrained(ROBERTA_PRETRAINED, config=robert_config_question)
        model_q.zero_grad();
        model_q.to(device);
        torch.cuda.empty_cache()
        dict_load = torch.load(path_join(ROBERTA_SAVED_MODEL_QUESTIONS[qa], 'Q_model_save_fold_{}.pt'.format(f)))
        model_q.load_state_dict(dict_load)
        out_q = predict_result_question(model_q, test_loader_q)

        model_a = RobertaForSequenceClassification_v2.from_pretrained(ROBERTA_PRETRAINED,config=robert_config_answer)
        model_a.zero_grad();
        model_a.to(device);
        torch.cuda.empty_cache()
        dict_load = torch.load(path_join(ROBERTA_SAVED_MODEL_ANSWERS[qa], 'A_model_save_fold_{}.pt'.format(f)))
        model_a.load_state_dict(dict_load)
        out_a = predict_result_answer(model_a, test_loader_a)
        
        preds = np.hstack((out_q,out_a))
        
        if POST_PROCESS:             
            # Round categories
            for l in class_labels_dict.keys():
                preds[:,class_labels_dict[l]] = pd.cut(preds[:,class_labels_dict[l]], train[l].nunique(), labels=train[l].value_counts().index.sort_values())

        test_predictions.append(preds)
    

In [ ]:
test_preds_roberta = np.mean(test_predictions,axis=0)
idx = np.argwhere(test.category!='CULTURE').flatten()
arr = 0.0001 * np.clip(np.random.normal(size=len(idx)),0.1,0.99)
test_preds_roberta[idx,19] = arr    

In [ ]:
class_labels = ['question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation']


submission = pd.read_csv('../input/google-quest-challenge/sample_submission.csv')
submission.iloc[:,1:] = test_preds_bert * 0.70 + test_preds_roberta*0.30


# Round categories
for l in class_labels:
    submission[l] = pd.cut(submission[l], train[l].nunique(), labels=train[l].value_counts().index.sort_values())

    
submission.to_csv('submission.csv', index=False)
submission.head() 
    